# Event Finder

The goal of this notebook is to develop a generalized event finder. Based on user selected variable and event definitions, this notebook will search all simulations to find these events.  

Inputs:

* Domain
* Variable
* Low and/or High thresholds
* Duration 
* Global Warming Level

**Runtime**: With the default settings, this notebook takes approximately 27 minutes to run from start to finish. Modifications to selections may increase the runtime.

**Troubleshooting**: Kernel crashing? Try loading a smaller spatial area by setting the `latitude` and `longitude` variables or using a `cached_area`.

## Make Selections

Before running this notebook, make the following selections for variables, domain, and GWLs.

In [1]:
from climakitae.core.constants import UNSET
from climakitae.new_core.user_interface import ClimateData

cd = ClimateData()

2026-02-14 01:27:59 - climakitae.new_core.user_interface - INFO - Initializing ClimateData interface
2026-02-14 01:27:59 - climakitae.new_core.dataset_factory - INFO - DatasetFactory initialized with 3 validators and 11 processors
2026-02-14 01:27:59 - climakitae.new_core.user_interface - INFO - ClimateData initialization successful
2026-02-14 01:27:59 - climakitae.new_core.user_interface - INFO - ✅ Ready to query!


### Selections option 1: Using data from cadcat catalog

1. Select a timescale - this will impact which downscaling methods are available and which variables are available.

In [2]:
timescale = "day"  # options are 1hr, day, mon

2. Select a variable - the code below displays which variables are available based on your timescale options

Here are the options for dynamically downscaled (WRF) variables for selected timescale:

In [3]:
cd.activity_id("WRF").table_id(timescale).show_variable_options()

2026-02-14 01:27:59 - climakitae.new_core.user_interface - INFO - Activity ID set to: WRF
2026-02-14 01:27:59 - climakitae.new_core.user_interface - INFO - Table ID set to: day
2026-02-14 01:27:59 - climakitae.new_core.user_interface - INFO - Variables (constrained by current query)::
2026-02-14 01:27:59 - climakitae.new_core.user_interface - INFO - ------------------------------------------
2026-02-14 01:27:59 - climakitae.new_core.user_interface - INFO - cape:                    Convective Available Potential Energy
2026-02-14 01:27:59 - climakitae.new_core.user_interface - INFO - cf:                      Capacity factor
2026-02-14 01:27:59 - climakitae.new_core.user_interface - INFO - cin:                     Convective Inhibition
2026-02-14 01:27:59 - climakitae.new_core.user_interface - INFO - dew_point:               No description available
2026-02-14 01:27:59 - climakitae.new_core.user_interface - INFO - effective_temp_index:    No description available
2026-02-14 01:27:59 - cl

Here are the options for statistically downscaled (LOCA2) variables for the selected timescale. This is a smaller set than the dynamiclly downscaled variables. Daily average humidity and air temperature are not listed, but can be selected in this notebook by using "tas" or "hurs" (daily max and min will be averaged).

In [4]:
cd.activity_id("LOCA2").table_id(timescale).show_variable_options()

2026-02-14 01:28:00 - climakitae.new_core.user_interface - INFO - Activity ID set to: LOCA2
2026-02-14 01:28:00 - climakitae.new_core.user_interface - INFO - Table ID set to: day
2026-02-14 01:28:00 - climakitae.new_core.user_interface - INFO - Variables (constrained by current query)::
2026-02-14 01:28:00 - climakitae.new_core.user_interface - INFO - ------------------------------------------
2026-02-14 01:28:00 - climakitae.new_core.user_interface - INFO - hursmax:    Maximum relative humidity
2026-02-14 01:28:00 - climakitae.new_core.user_interface - INFO - hursmin:    Minimum relative humidity
2026-02-14 01:28:00 - climakitae.new_core.user_interface - INFO - huss:       Specific humidity at 2m
2026-02-14 01:28:00 - climakitae.new_core.user_interface - INFO - pr:         Precipitation (total)
2026-02-14 01:28:00 - climakitae.new_core.user_interface - INFO - rsds:       Shortwave flux at the surface
2026-02-14 01:28:00 - climakitae.new_core.user_interface - INFO - tasmax:     Maximum

In [5]:
# Make selection from lists above

# Dynamically downscaled variable (required)
event_variable_wrf = "prec"

# Set units
event_units_wrf = "inches/d"

# Statistically downscaled variable (optional)
# For "Relative humidity" use "hurs"
# For "Air Temperature at 2m" use "tas"
# For no variable use UNSET (no quotes)
event_variable_loca2 = "pr"

# Set units
event_units_loca2 = "inches"

3. Define Event - Select a method for establishing the event

In [6]:
# define the function that will be used to calculate the event threshold
event_threshold_function = (
    "1 in x"  # options are "percentile", "1 in x", or "actual value"
)
event_threshold_direction = (
    ">"  # look for events above or below the result from the threshold function
)
event_threshold_function_params = (
    10  # if 1 in X include X, if quantile, include quantile (0.25, 0.5, etc.)
)
event_duration = 1  # how many consecutive events need to be observed to be a hit

4. Select a baseline and future GWL

In [7]:
# current options 0.8, 1.0, 1.2, 1.5, 2.0, 2.5, 3.0, 4.0
baseline_gwl = 1.0
future_gwl = 2.0

5. Select Spatial Domain - A spatial domain can be selected via a lat/lon box, a cached area, or a user-provided shapefile. Selections can be made in the cell below.

For example, someone studying the Pajaro watershed might have a shapefile path that looks like this:
```
# User selection
spatial_domain = "Pajaro"
shapefile_filename = "PajaroRiverWatershed.zip"

# Other variables left unset:
lats = ()
lons = ()
cached_area = UNSET
```

In [8]:
# name will appear in title
spatial_domain = "CA"

# can define lat/lon box OR cached area OR shapefile
lats = ()  # () to leave undefined
lons = ()
cached_area = "Santa Cruz County"  # UNSET for no cached area

# edit filename below for shapefile to use
shapefile_filename = UNSET  # use UNSET for no shapefile

6. Select Resolution - select the the model resolution (3km, 9km, 45 km)

In [9]:
resolution = "d03"  # Options are d01 (45 km), d02 (9 km), or d03 (3 km)

7. Aggregation Method - select a method to aggregate spatially

In [10]:
aggregation = "median"  # options are mean, median, sum, min, max

8. Processes - choices for warming levels, units, and clipping are assembled into a dictionary of processes to apply to data selection.

In [11]:
# this dictionary is passed to the ClimateData catalog call when pulling data
process_dict = {
    "warming_level": {
        "warming_levels": [baseline_gwl],
    },
    "metric_calc": {"metric": aggregation, "dim": ["y", "x"]},
}

# add clipping selection
if (len(lats) > 0) and (len(lons) > 0):
    process_dict["clip"] = (lats, lons)
elif cached_area is not UNSET:
    process_dict["clip"] = [cached_area]
elif shapefile_filename is not UNSET:
    process_dict["clip"] = shapefile_filename
else:
    print("Process 'clip' unset.")

### Selections option 2: Upload custom timeseries

Users may provide their own timeseries data to the event finder. To use this option, set the filenames and delete the `'''` characters in the code block below. These settings will take precedent over the settings in Selections Option 1.

Formatting: This data should be stored as an xarray Dataset with dimensions `sim`, `warming levels` and `time`. The variable names must match `event_variable_wrf` and `event_variable_loca2`. Data must be converted to desired units and must be spatially aggregated before use.

In [12]:
# Set paths to data file(s) here:
wrf_filename = UNSET
loca2_filename = UNSET

"""
# These default settings are for a Santa Ana Winds case
# Dynamically downscaled variable
event_variable_wrf = "MSLP_gradient"

# WRF data units
event_units_wrf = "hPa"

# Statistically downscaled variable
event_variable_loca2 = UNSET

# LOCA2 data units
event_units_loca2 = UNSET

# Timescale will appear in file names
timescale = "1hr"

# Domain will appear in title
spatial_domain = "Santa Ana wind"

# warming levels (must be present in file)
baseline_gwl = 0.8
future_gwl = 2.0

# define the function that will be used to calculate the event threshold
event_threshold_function = (
    "actual value"  # options are "percentile", "1 in x", or "actual value"
)
event_threshold_direction = (
    "<"  # look for events above or below the result from the threshold function
)
event_threshold_function_params = (
    -5  # if 1 in X include X, if quantile, include quantile (0.25, 0.5, etc.)
)
event_duration = 1  # how many consecutive events need to be observed to be a hit
"""

'\n# These default settings are for a Santa Ana Winds case\n# Dynamically downscaled variable\nevent_variable_wrf = "MSLP_gradient"\n\n# WRF data units\nevent_units_wrf = "hPa"\n\n# Statistically downscaled variable\nevent_variable_loca2 = UNSET\n\n# LOCA2 data units\nevent_units_loca2 = UNSET\n\n# Timescale will appear in file names\ntimescale = "1hr"\n\n# Domain will appear in title\nspatial_domain = "Santa Ana wind"\n\n# warming levels (must be present in file)\nbaseline_gwl = 0.8\nfuture_gwl = 2.0\n\n# define the function that will be used to calculate the event threshold\nevent_threshold_function = (\n    "actual value"  # options are "percentile", "1 in x", or "actual value"\n)\nevent_threshold_direction = (\n    "<"  # look for events above or below the result from the threshold function\n)\nevent_threshold_function_params = (\n    -5  # if 1 in X include X, if quantile, include quantile (0.25, 0.5, etc.)\n)\nevent_duration = 1  # how many consecutive events need to be observed 

## Import libraries & define functions

In [13]:
import climakitae as ck

from climakitae.util.utils import add_dummy_time_to_wl
from climakitae.explore.threshold_tools import (
    get_block_maxima,
    get_return_value,
)
from climakitae.core.data_load import load

import xarray as xr
import pandas as pd
import geopandas as gpd
from shapely.geometry import mapping
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

## Step 1: Define Event 

Use the baseline GWL to define the event threshold

### Pull Baseline Data

#### Dynamical Downscaling (WRF)

In [14]:
if wrf_filename is not UNSET:
    wrf_data = xr.open_dataset(wrf_filename)
    wrf_data = wrf_data.sel(warming_level=baseline_gwl)
    wrf_data = load(wrf_data, progress_bar=True)
else:
    process_dict.update({"convert_units": event_units_wrf})
    wrf_data = (
        cd.catalog("cadcat")
        .activity_id("WRF")
        .institution_id("UCLA")
        .table_id(timescale)
        .grid_label(resolution)
        .variable(event_variable_wrf)
        .processes(process_dict)
        .get()
    )

    # add a dummy time variable
    wrf_data = add_dummy_time_to_wl(wrf_data)

    # load data
    wrf_data = load(wrf_data, progress_bar=True).squeeze(dim="warming_level")

2026-02-14 01:28:00 - climakitae.new_core.user_interface - INFO - Catalog set to: cadcat
2026-02-14 01:28:00 - climakitae.new_core.user_interface - INFO - Activity ID set to: WRF
2026-02-14 01:28:00 - climakitae.new_core.user_interface - INFO - Institution ID set to: UCLA
2026-02-14 01:28:00 - climakitae.new_core.user_interface - INFO - Table ID set to: day
2026-02-14 01:28:00 - climakitae.new_core.user_interface - INFO - Grid label set to: d03
2026-02-14 01:28:00 - climakitae.new_core.user_interface - INFO - Variable set to: prec
2026-02-14 01:28:00 - climakitae.new_core.user_interface - INFO - Processes set: 4 operations configured
2026-02-14 01:28:00 - climakitae.new_core.user_interface - INFO - Starting data retrieval with query: {'catalog': 'cadcat', 'installation': UNSET, 'activity_id': 'WRF', 'institution_id': 'UCLA', 'source_id': UNSET, 'experiment_id': UNSET, 'table_id': 'day', 'grid_label': 'd03', 'variable_id': 'prec', 'station_id': UNSET, 'network_id': UNSET, 'processes': {


Incomplete warming level for WRF.UCLA.EC-Earth3-Veg.ssp370.day.d03.r1i1p1f1 at 1.0C. 
Skipping this warming level.
  warnings.warn(



2026-02-14 01:28:10 - climakitae.new_core.processors.warming_level - WARNING - 

Incomplete warming level for WRF.UCLA.EC-Earth3-Veg.ssp370.day.d03.r1i1p1f1 at 1.0C. 
Skipping this warming level.
2026-02-14 01:28:10 - climakitae.new_core.processors.warming_level - WARNING - No valid slices found for WRF.UCLA.EC-Earth3-Veg.ssp370.day.d03.r1i1p1f1. Ensure the warming level times table is correctly configured.
2026-02-14 01:28:11 - climakitae.new_core.processors.concatenate - INFO - Concatenated datasets along 'sim' dimension.
2026-02-14 01:28:12 - climakitae.new_core.processors.convert_units - INFO - Units successfully converted to 'inches/d'
2026-02-14 01:28:12 - climakitae.new_core.processors.update_attributes - INFO - UpdateAttributes applied to result; added 9 attributes
2026-02-14 01:28:12 - climakitae.new_core.dataset - INFO - All processing steps completed successfully
2026-02-14 01:28:12 - climakitae.new_core.user_interface - INFO - ✅ Data retrieval successful!
Processing data to

#### Statistical Downscaling (LOCA2)

In [15]:
if event_variable_loca2 is not UNSET:
    if loca2_filename is not UNSET:
        loca_data = xr.open_dataset(loca2_filename)
        loca_data = loca_data.sel(warming_level=baseline_gwl)
        loca_data = load(loca_data, progress_bar=True)
    else:
        loca_variable = [event_variable_loca2]
        # temperature
        if loca_variable[0] == "tas":
            loca_variable = ["tasmax", "tasmin"]
        # relative humidity
        if loca_variable[0] == "hurs":
            loca_variable = ["hursmax", "hursmin"]

        process_dict.update({"convert_units": event_units_loca2})
        process_dict["metric_calc"].update({"dim": ["lat", "lon"]})

        # statistical - loca2 downscaling xvariable
        loca_data = [
            cd.catalog("cadcat")
            .activity_id("LOCA2")
            .table_id(timescale)
            .grid_label(resolution)
            .variable(GETVAR)
            .processes(process_dict)
            .get()
            for GETVAR in loca_variable
        ]

        # if we need to average
        if event_variable_loca2 in ["tas", "hurs"]:
            loca_data = (
                loca_data[0][loca_variable[0]] + loca_data[1][loca_variable[1]]
            ) / 2
            loca_data = loca_data.to_dataset(name=event_variable_loca2)
            loca_data.attrs["frequency"] = timescale

        # unlist if needed
        if type(loca_data) == list:
            loca_data = loca_data[0]

        # add a dummy time variable
        loca_data = add_dummy_time_to_wl(loca_data)

        # reset to WRF values
        process_dict["metric_calc"].update({"dim": ["x", "y"]})

        # load data
        loca_data = load(loca_data, progress_bar=True).squeeze(dim="warming_level")
else:
    # set to wrf
    loca_data = wrf_data
    # replace with all NaNs
    loca_data = loca_data * np.nan

2026-02-14 01:28:19 - climakitae.new_core.user_interface - INFO - Catalog set to: cadcat
2026-02-14 01:28:19 - climakitae.new_core.user_interface - INFO - Activity ID set to: LOCA2
2026-02-14 01:28:19 - climakitae.new_core.user_interface - INFO - Table ID set to: day
2026-02-14 01:28:19 - climakitae.new_core.user_interface - INFO - Grid label set to: d03
2026-02-14 01:28:19 - climakitae.new_core.user_interface - INFO - Variable set to: pr
2026-02-14 01:28:19 - climakitae.new_core.user_interface - INFO - Processes set: 4 operations configured
2026-02-14 01:28:19 - climakitae.new_core.user_interface - INFO - Starting data retrieval with query: {'catalog': 'cadcat', 'installation': UNSET, 'activity_id': 'LOCA2', 'institution_id': UNSET, 'source_id': UNSET, 'experiment_id': UNSET, 'table_id': 'day', 'grid_label': 'd03', 'variable_id': 'pr', 'station_id': UNSET, 'network_id': UNSET, 'processes': {'warming_level': {'warming_levels': [1.0]}, 'metric_calc': {'metric': 'median', 'dim': ['lat', 

### Define Event

In [16]:
# determine what method to define event
if event_threshold_function == "actual value":
    loca_threshold = event_threshold_function_params
    wrf_threshold = event_threshold_function_params

if event_threshold_function == "percentile":
    # calculate the percentile
    loca_threshold = (
        loca_data[event_variable_loca2]
        .quantile(dim="time", q=event_threshold_function_params)
        .squeeze()
    )
    wrf_threshold = (
        wrf_data[event_variable_wrf]
        .quantile(dim="time", q=event_threshold_function_params)
        .squeeze()
    )

# if 1 in x
if event_threshold_function == "1 in x":
    if event_threshold_direction == ">":
        extreme_type = "max"
    else:
        extreme_type = "min"
    # initialize
    loca_list = []
    for sim in loca_data.sim:
        loca_ams = get_block_maxima(
            loca_data[event_variable_loca2].sel({"sim": sim}),
            check_ess=False,
            extremes_type=extreme_type,
        )
        loca_rv = get_return_value(
            loca_ams,
            return_period=event_threshold_function_params,
            multiple_points=False,
        )
        loca_rv = loca_rv.expand_dims(dim="sim")
        loca_list.append(loca_rv)
    loca_threshold = xr.concat(loca_list, loca_data.sim)
    loca_threshold = loca_threshold["return_value"].squeeze()
    # intalize
    wrf_list = []
    for sim in wrf_data.sim:
        wrf_ams = get_block_maxima(
            wrf_data[event_variable_wrf].sel({"sim": sim}),
            check_ess=False,
            extremes_type=extreme_type,
        )
        wrf_rv = get_return_value(
            wrf_ams,
            return_period=event_threshold_function_params,
            multiple_points=False,
        )
        wrf_rv = wrf_rv.expand_dims(dim="sim")
        wrf_list.append(wrf_rv)
    wrf_threshold = xr.concat(wrf_list, wrf_data.sim)
    wrf_threshold = wrf_threshold["return_value"].squeeze()

In [17]:
# remove baseline data to free up memory
del loca_data
del wrf_data

In [17]:
loca_data

<xarray.Dataset> Size: 6MB
Dimensions:        (sim: 129, time: 10950)
Coordinates:
  * sim            (sim) object 1kB 'LOCA2_UCSD_EC-Earth3-Veg_ssp370_day_d03_...
  * time           (time) datetime64[ns] 88kB 2000-01-01 ... 2029-12-31
    warming_level  float64 8B 1.0
    centered_year  (sim) float64 1kB 1.988e+03 1.999e+03 ... 2.005e+03 2.012e+03
    spatial_ref    int64 8B 0
Data variables:
    pr             (sim, time) float32 6MB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
Attributes: (12/94)
    Conventions:                         CF-1.7 CMIP-6.2
    ID_loca_routines_module:             $Id: loca_routines_module.F90,v 1.13...
    SIOCRD_netCDF_Version:               1.0
    SOURCE_loca_routines_module:         $Source: /home6/dwpierc2/src/mine/lo...
    activity_id:                         CMIP
    bias_correction:                     downscaling via PresRat, Pierce et a...
    ...                                  ...
    convert_units:                       Process 'convert_units' applied to t...
    update_attributes:                   Process 'update_attributes' applied ...
    filter_unadjusted_models:            yes
    drop_leap_days:                      Leap days (February 29) have been re...
    concat:                              Process 'concat' applied to the data...
    warming_level_simple:                Process 'warming_level_simple' appli...

In [18]:
wrf_data

<xarray.Dataset> Size: 263kB
Dimensions:            (sim: 4, time: 10950)
Coordinates:
  * sim                (sim) object 32B 'WRF_UCLA_TaiESM1_ssp370_day_d03_r1i1...
  * time               (time) datetime64[ns] 88kB 2000-01-01 ... 2029-12-31
    warming_level      float64 8B 1.0
    centered_year      (sim) float64 32B 2.017e+03 2.023e+03 2.003e+03 2.006e+03
    Lambert_Conformal  int64 8B 0
Data variables:
    prec               (sim, time) float32 175kB 0.02497 0.0 ... 0.02953 0.02803
Attributes: (12/187)
    ADAPT_DT_MAX:                     24.0
    ADAPT_DT_MIN:                     12.0
    ADAPT_DT_START:                   18.0
    AERCU_FCT:                        1.0
    AERCU_OPT:                        0
    AER_ANGEXP_OPT:                   1
    ...                               ...
    convert_units:                    Process 'convert_units' applied to the ...
    update_attributes:                Process 'update_attributes' applied to ...
    filter_unadjusted_models:         yes
    drop_leap_days:                   Leap days (February 29) have been remov...
    concat:                           Process 'concat' applied to the data. T...
    warming_level_simple:             Process 'warming_level_simple' applied ...

In [19]:
wrf_threshold

<xarray.DataArray 'return_value' (sim: 4)> Size: 32B
array([4.57739, 4.04058, 4.9816 , 3.56044])
Coordinates:
  * sim                (sim) object 32B 'WRF_UCLA_TaiESM1_ssp370_day_d03_r1i1...
    warming_level      float64 8B 1.0
    centered_year      (sim) float64 32B 2.017e+03 2.023e+03 2.003e+03 2.006e+03
    Lambert_Conformal  int64 8B 0
    one_in_x           int64 8B 10
Attributes:
    long_name:                        total precipitation
    units:                            inches/d
    extremes type:                    maxima
    duration:                         UNSET
    groupby:                          UNSET
    grouped_duration:                 UNSET
    extreme_value_extraction_method:  block maxima
    block_size:                       1 year
    timeseries_type:                  block max series
    return period:                    1-in-[10]-year event

In [20]:
loca_threshold

<xarray.DataArray 'return_value' (sim: 129)> Size: 1kB
array([4.5329 , 4.78528, 4.0105 , 4.90149, 3.94414, 4.85345, 5.00783,
       3.99884, 4.2101 , 5.58378, 4.13114, 4.62571, 4.57342, 4.67404,
       5.09607, 4.96256, 4.54953, 4.05101, 4.21181, 4.62413, 4.12933,
       4.93381, 3.75076, 4.88368, 4.01964, 4.81935, 4.94178, 4.05101,
       4.21181, 4.62413, 4.06123, 4.55582, 4.5329 , 4.78528, 4.0105 ,
       4.71662, 5.57679, 4.31469, 4.62484, 4.1164 , 4.65304, 4.18146,
       4.22191, 4.55582, 4.29957, 4.25992, 4.2101 , 5.58378, 4.13114,
       4.71243, 4.42667, 4.2101 , 5.58378, 4.13114, 4.62598, 4.57342,
       4.11732, 4.31645, 3.95364, 4.41615, 4.36019, 5.50261, 4.83147,
       4.54092, 4.20457, 4.13033, 5.50261, 5.1121 , 4.05434, 5.50261,
       4.98028, 5.13742, 4.4561 , 4.57182, 4.08333, 4.40684, 4.59527,
       4.5752 , 4.05101, 4.21181, 4.62413, 4.46609, 4.71263, 4.44291,
       4.51573, 4.55582, 5.0405 , 4.67218, 4.66384, 4.71715, 3.65114,
       4.10681, 5.14595, 4.12236, 4.76287, 5.04839, 4.34967, 4.65002,
       5.06175, 4.65711, 4.95312, 5.27811, 4.36323, 5.0434 , 4.34757,
       3.90213, 3.70542, 4.92909, 5.05324, 4.64161, 5.3947 , 4.58247,
       4.35954, 3.9991 , 4.12705, 4.97184, 4.50029, 4.63489, 4.19926,
       4.28381, 4.5329 , 4.78528, 4.0105 , 4.74939, 4.47748, 4.68144,
       4.59596, 4.52694, 4.53637])
Coordinates:
  * sim            (sim) object 1kB 'LOCA2_UCSD_EC-Earth3-Veg_ssp370_day_d03_...
    warming_level  float64 8B 1.0
    centered_year  (sim) float64 1kB 1.988e+03 1.999e+03 ... 2.005e+03 2.012e+03
    spatial_ref    int64 8B 0
    one_in_x       int64 8B 10
Attributes:
    long_name:                        pr
    units:                            inches
    extremes type:                    maxima
    duration:                         UNSET
    groupby:                          UNSET
    grouped_duration:                 UNSET
    extreme_value_extraction_method:  block maxima
    block_size:                       1 year
    timeseries_type:                  block max series
    return period:                    1-in-[10]-year event

## Step 2: Estimate Event 'Hits'

Use the future GWL to estimate event hits

### Pull Future GWL Data

#### Dynamical Downscaling (WRF)

In [21]:
# WRF downscaling
if wrf_filename is not UNSET:
    wrf_data = xr.open_dataset(wrf_filename)
    wrf_data = wrf_data.sel(warming_level=future_gwl)
    wrf_data = load(wrf_data, progress_bar=True)
else:
    process_dict.update({"warming_level": {"warming_levels": [future_gwl]}})
    process_dict.update({"convert_units": event_units_wrf})

    wrf_data = (
        cd.catalog("cadcat")
        .activity_id("WRF")
        .institution_id("UCLA")
        .table_id(timescale)
        .grid_label(resolution)
        .variable(event_variable_wrf)
        .processes(process_dict)
        .get()
    )

    # add a dummy time variable
    wrf_data = add_dummy_time_to_wl(wrf_data)

    # If simulation is missing in baseline dataset,
    # drop from future.
    wrf_data = wrf_data.sel({"sim": wrf_threshold.sim.data})

    # load data
    wrf_data = load(wrf_data, progress_bar=True).squeeze(dim="warming_level")

2026-02-14 01:42:15 - climakitae.new_core.user_interface - INFO - Catalog set to: cadcat
2026-02-14 01:42:15 - climakitae.new_core.user_interface - INFO - Activity ID set to: WRF
2026-02-14 01:42:15 - climakitae.new_core.user_interface - INFO - Institution ID set to: UCLA
2026-02-14 01:42:15 - climakitae.new_core.user_interface - INFO - Table ID set to: day
2026-02-14 01:42:15 - climakitae.new_core.user_interface - INFO - Grid label set to: d03
2026-02-14 01:42:15 - climakitae.new_core.user_interface - INFO - Variable set to: prec
2026-02-14 01:42:15 - climakitae.new_core.user_interface - INFO - Processes set: 4 operations configured
2026-02-14 01:42:15 - climakitae.new_core.user_interface - INFO - Starting data retrieval with query: {'catalog': 'cadcat', 'installation': UNSET, 'activity_id': 'WRF', 'institution_id': 'UCLA', 'source_id': UNSET, 'experiment_id': UNSET, 'table_id': 'day', 'grid_label': 'd03', 'variable_id': 'prec', 'station_id': UNSET, 'network_id': UNSET, 'processes': {

#### Statistical Downscaling (LOCA2)

In [ ]:
if event_variable_loca2 is not UNSET:
    if loca2_filename is not UNSET:
        loca_data = xr.open_dataset(loca2_filename)
        loca_data = loca_data.sel(warming_level=future_gwl)
        loca_data = load(loca_data, progress_bar=True)
    else:
        # grab correct variable name for LOCA2
        loca_variable = [event_variable_loca2]
        # temperature
        if loca_variable[0] == "tas":
            loca_variable = ["tasmax", "tasmin"]
        # relative humidity
        if loca_variable[0] == "hurs":
            loca_variable = ["hursmax", "hursmin"]

        process_dict.update({"convert_units": event_units_loca2})
        process_dict["metric_calc"].update({"dim": ["lat", "lon"]})

        # statistical - loca2 downscaling xvariable
        loca_data = [
            cd.catalog("cadcat")
            .activity_id("LOCA2")
            .table_id(timescale)
            .grid_label(resolution)
            .variable(GETVAR)
            .processes(process_dict)
            .get()
            for GETVAR in loca_variable
        ]

        # if we need to average
        if event_variable_loca2 in ["tas", "hurs"]:
            loca_data = (
                loca_data[0][loca_variable[0]] + loca_data[1][loca_variable[1]]
            ) / 2
            loca_data = loca_data.to_dataset(name=event_variable_loca2)
            loca_data.attrs["frequency"] = timescale

        # pull data from list if needed
        if type(loca_data) == list:
            loca_data = loca_data[0]

        # add a dummy time variable
        loca_data = add_dummy_time_to_wl(loca_data)

        # If simulation is missing in baseline dataset,
        # drop from future.
        loca_data = loca_data.sel({"sim": loca_threshold.sim.data})

        # reset to WRF values
        process_dict["metric_calc"].update({"dim": ["x", "y"]})

        # load data
        loca_data = load(loca_data, progress_bar=True).squeeze(dim="warming_level")
else:
    # set to wrf
    loca_data = wrf_data
    # replace with all NaNs
    loca_data = loca_data * np.nan

2026-02-14 01:42:28 - climakitae.new_core.user_interface - INFO - Catalog set to: cadcat
2026-02-14 01:42:28 - climakitae.new_core.user_interface - INFO - Activity ID set to: LOCA2
2026-02-14 01:42:28 - climakitae.new_core.user_interface - INFO - Table ID set to: day
2026-02-14 01:42:28 - climakitae.new_core.user_interface - INFO - Grid label set to: d03
2026-02-14 01:42:28 - climakitae.new_core.user_interface - INFO - Variable set to: pr
2026-02-14 01:42:28 - climakitae.new_core.user_interface - INFO - Processes set: 4 operations configured
2026-02-14 01:42:28 - climakitae.new_core.user_interface - INFO - Starting data retrieval with query: {'catalog': 'cadcat', 'installation': UNSET, 'activity_id': 'LOCA2', 'institution_id': UNSET, 'source_id': UNSET, 'experiment_id': UNSET, 'table_id': 'day', 'grid_label': 'd03', 'variable_id': 'pr', 'station_id': UNSET, 'network_id': UNSET, 'processes': {'warming_level': {'warming_levels': [2.0]}, 'metric_calc': {'metric': 'median', 'dim': ['lat', 

### Count Hits

In [ ]:
### create a 'hit' for in events
# initialize an array to fill
wrfEventHit = np.zeros(wrf_data[event_variable_wrf].shape)
if event_variable_loca2 is not UNSET:
    locaEventHit = np.zeros(loca_data[event_variable_loca2].shape)
else:
    locaEventHit = wrfEventHit

if len(wrf_data.dims) == 2:
    print("2D")
    # loop through each time stamp
    dim3 = UNSET
    for itime in range(0, len(wrf_data["time"])):
        # look at this time stamp + duration
        timeIndex = list(range(itime, (itime + event_duration)))
        # remove any values that are greater than our time
        timeIndex = [x for x in timeIndex if x < len(wrf_data["time"])]
    
        ### Start with WRF
        # pull out the data to test
        testWRFData = wrf_data[event_variable_wrf][:, timeIndex]
    
        # for each simulation, check if the values are greater or less than the threshold
        if event_threshold_direction == "<":
            wrfCounts = (testWRFData < wrf_threshold).sum(dim="time")
        if event_threshold_direction == ">":
            wrfCounts = (testWRFData > wrf_threshold).sum(dim="time")
    
        # if you have greater than or equal to duration count as a 'hit'
        wrfHitIndex = wrfCounts >= event_duration
    
        # now save hits
        wrfEventHit[wrfHitIndex, itime] = 1
    
        ### Move to Loca
        if event_variable_loca2 is not UNSET:
            # pull out the data to test
            testLOCAData = loca_data[event_variable_loca2][:, timeIndex]
    
            # for each simulation, check if the values are greater or less than the threshold
            if event_threshold_direction == "<":
                locaCounts = (testLOCAData < loca_threshold).sum(dim="time")
            if event_threshold_direction == ">":
                locaCounts = (testLOCAData > loca_threshold).sum(dim="time")
    
            # if you have greater than or equal to duration count as a 'hit'
            locaHitIndex = locaCounts >= event_duration
    
            # now save hits
            locaEventHit[locaHitIndex, itime] = 1
elif len(wrf_data.dims) == 3:
    print("3D")
    # Get name of third dimension
    dim3 = [x for x in list(wrf_data.dims.keys()) if x not in ["sim","time"]][0]
    
    # loop through each time stamp
    for itime in range(0, len(wrf_data["time"])):
        for idim in range(0, len(wrf_data[dim3])):
            # look at this time stamp + duration
            timeIndex = list(range(itime, (itime + event_duration)))
            # remove any values that are greater than our time
            timeIndex = [x for x in timeIndex if x < len(wrf_data["time"])]
        
            ### Start with WRF
            # pull out the data to test
            testWRFData = wrf_data[event_variable_wrf][:, timeIndex, idim]
        
            # for each simulation, check if the values are greater or less than the threshold
            if event_threshold_direction == "<":
                wrfCounts = (testWRFData < wrf_threshold).sum(dim="time")
            if event_threshold_direction == ">":
                wrfCounts = (testWRFData > wrf_threshold).sum(dim="time")
        
            # if you have greater than or equal to duration count as a 'hit'
            wrfHitIndex = wrfCounts >= event_duration
        
            # now save hits
            wrfEventHit[wrfHitIndex, itime, idim] = 1
        
            ### Move to Loca
            if event_variable_loca2 is not UNSET:
                # pull out the data to test
                testLOCAData = loca_data[event_variable_loca2][:, timeIndex, idim]
        
                # for each simulation, check if the values are greater or less than the threshold
                if event_threshold_direction == "<":
                    locaCounts = (testLOCAData < loca_threshold).sum(dim="time")
                if event_threshold_direction == ">":
                    locaCounts = (testLOCAData > loca_threshold).sum(dim="time")
        
                # if you have greater than or equal to duration count as a 'hit'
                locaHitIndex = locaCounts >= event_duration
        
                # now save hits
                locaEventHit[locaHitIndex, itime, idim] = 1

In [ ]:
# now add hits to xarray
wrf_data = wrf_data.assign_coords(event_hit=(wrf_data.dims, wrfEventHit))
loca_data = loca_data.assign_coords(event_hit=(wrf_data.dims, locaEventHit))

# Rename so that there's only a single variable in the final dataframe
if event_variable_loca2 is not UNSET:
    loca_data = loca_data.rename({event_variable_loca2: event_variable_wrf})

## Prep for Plotting

In [ ]:
wsims = wrf_data.sim.values.tolist()
if not isinstance(wsims,list):
    wsims = [wrf_data.sim[0].data.item()]
wrf_models = [s.split("_")[2] for s in wsims]
wrf_data = wrf_data.assign_coords(models=("sim", wrf_models))

# create a realization variable
wrf_realization = [s.split("_")[3] + "_" + s.split("_")[6] for s in wsims]
wrf_data = wrf_data.assign_coords(realization=("sim", wrf_realization))

# create a data frame to make plotting easier
wrfDF = wrf_data.to_dataframe().reset_index()

# Create a downscaling name
wrfDF["downscaling"] = "wrf"

In [ ]:
if event_variable_loca2 is not UNSET:
    # create a 'model' variable
    lsims = loca_data.sim.values.tolist()
    loca_models = [s.split("_")[2] for s in lsims]
    loca_data = loca_data.assign_coords(models=("sim", loca_models))

    # create a realization variable
    loca_realization = [s.split("_")[3] + "_" + s.split("_")[6] for s in lsims]
    loca_data = loca_data.assign_coords(realization=("sim", loca_realization))

    # create a data frame to make plotting easier
    locaDF = loca_data.to_dataframe().reset_index()

    locaDF["downscaling"] = "loca2"
else:
    # Just an empty dataframe
    locaDF = pd.DataFrame().reindex_like(wrfDF)

In [ ]:
# combine into one
if event_variable_loca2 is not UNSET:
    finalDF = pd.concat([locaDF, wrfDF], keys=wrfDF.keys())
else:
    finalDF = wrfDF

## Step 3: Plot Results (Yay!)

In [ ]:
# plot all the data
def annotate(data, **kws):
    ax = plt.gca()
    ax.scatter(
        data.time.where(data.event_hit == 1),
        data[event_variable_wrf].where(data.event_hit == 1),
        color="r",
    )

figures = {}
if dim3 is not UNSET:
    dim3_values = wrf_data[dim3].data
    for value in dim3_values:
        g1 = sns.relplot(
            data=finalDF[finalDF[dim3]==value],
            x="time",
            y=event_variable_wrf,
            col="sim",
            color="grey",
            col_wrap=4,
            kind="line",
            height=5,
            aspect=1,
            facet_kws=dict(sharex=False),
        )
        g1.map_dataframe(annotate)
        figures[value] = g1
else: # Just sim and time
    g1 = sns.relplot(
        data=finalDF,
        x="time",
        y=event_variable_wrf,
        col="sim",
        color="grey",
        col_wrap=4,
        kind="line",
        height=5,
        aspect=1,
        facet_kws=dict(sharex=False),
    )
    g1.map_dataframe(annotate)
    figures["none"] = g1

In [ ]:
# Save the figure to a file
if event_threshold_function == "1 in x":
    event_name = f"1_in_{event_threshold_function_params}"
elif event_threshold_function == "percentile":
    event_name = f"{event_threshold_function_params}_percentile"
else:
    event_name = f"{event_threshold_function_params}_{event_units_wrf}"
for value,fig in figures.items():
    dim_value = f"_{value}" if value != "none" else ""
    fig.savefig(
        f"event_finder_{event_variable_wrf}_under_{future_gwl}gwl_with_{event_name}_{event_duration}{timescale}duration_{spatial_domain}{dim_value}.jpeg".replace(
            " ", "_"
        )
    )

In [ ]:
# save event data
finalDF.to_parquet(
    f"event_{event_variable_wrf}_under_{future_gwl}gwl_with_{event_name}_{event_duration}{timescale}duration_{spatial_domain}".replace(
        " ", "_"
    )
)